In [ ]:
pip install duckduckgo-search

In [ ]:
pip install sentence-transformers

In [ ]:
!pip install -q "accelerate>=0.16.0,<1" "transformers[torch]>=4.28.1,<5" "torch>=1.13.1,<2"

In [ ]:
pip install accelerate

In [13]:
import time

In [6]:
import requests
import time
from bs4 import BeautifulSoup
from googlesearch import search
from difflib import SequenceMatcher
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
import torch

See: https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad

In [7]:
# Function to extract text from webpage using BeautifulSoup
def get_text(website_urls):
    texts = []
    for url in website_urls:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            texts.append(soup.get_text())
    return [text.lower() for text in texts]

# Function to perform Google search and get URLs
def google_search(query):
    return search(query, num=3, stop=5, pause=2)


In [8]:
import torch
dolly_pipeline = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
# Getting answer form LLM
def get_completion_dolly(input):
  prompt = f"User: {input}\nResponse from Dolly-v2-3b: "
  dolly_response = dolly_pipeline(prompt, max_new_tokens=500)
  return dolly_response[0]["generated_text"]

In [10]:
# Function to detect scams
def detect_scams(user_statement, famous_person):
    # Perform Google search, you can use other search engines like duckduckgo as well
    search_results = google_search(f'"{user_statement}" "{famous_person}"')

    if search_results:
        # Get text content from search results webpages
        texts = get_text(search_results)

    prompt = f"Did {famous_person} really say {user_statement} or is it fraud based on {texts}"

    #
    #s = time.time()
    output = get_completion_dolly(prompt)

    return output

In [11]:
# Example usage
user_statement = input("Enter your statement: ")
famous_person = input("Enter the name of the famous person: ")
detect_scams(user_statement, famous_person)

Enter your statement: buy le creuset cookware set
Enter the name of the famous person: taylor swift


'Taylor Swift said both in the same teleconference, she did not say buy Le Creuset, this is a fraud.'